In [1]:
# install huggingface transformers
!pip install transformers==4.11.2

     |████████████████████████████████| 2.9 MB 226 kB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 397 kB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.0
    Uninstalling transformers-4.20.0:
      Successfully uninstalled transformers-4.20.0


In [2]:
# load pretrained model
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
asr_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# get a single audio file
!wget https://dldata-public.s3.us-east-2.amazonaws.com/1919-142785-0028.wav

--2022-06-19 18:02:43--  https://dldata-public.s3.us-east-2.amazonaws.com/1919-142785-0028.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.104.24
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.104.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165164 (161K) [audio/wav]
Saving to: ‘1919-142785-0028.wav’

1919-142785-0028.wa 100%[===================>] 161.29K   218KB/s    in 0.7s    

2022-06-19 18:02:45 (218 KB/s) - ‘1919-142785-0028.wav’ saved [165164/165164]



In [5]:
# transcribe audio to logits over characters
import soundfile as sf
arr, _ = sf.read('1919-142785-0028.wav')
input_values = asr_processor(arr, return_tensors="pt", sampling_rate=16000).input_values  # Batch size 1
logits = asr_model(input_values).logits.cpu().detach().numpy()[0]

/home/manju/anaconda3/lib/python3.9/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  tensor = as_tensor(value)


In [6]:
# look at the alphabet of our model defining the labels for the logit matrix we just calculated
print(asr_processor.tokenizer.get_vocab())

{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, 'E': 5, 'T': 6, 'A': 7, 'O': 8, 'N': 9, 'I': 10, 'H': 11, 'S': 12, 'R': 13, 'D': 14, 'L': 15, 'U': 16, 'M': 17, 'W': 18, 'C': 19, 'F': 20, 'G': 21, 'Y': 22, 'P': 23, 'B': 24, 'V': 25, 'K': 26, "'": 27, 'X': 28, 'J': 29, 'Q': 30, 'Z': 31}


In [16]:
from pyctcdecode import build_ctcdecoder
vocab_list = list(asr_processor.tokenizer.get_vocab())
decoder = build_ctcdecoder(vocab_list)
decoder.decode(logits)

Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


'BOIL THEM BEFORE THEY ARE PUT INTO THE SOUP OR OTHER DISH THEY MAY BE INTENDED FOR'